In [ ]:
%matplotlib qt5
from hyperspy import api as hs
import numpy as np
import matplotlib.pyplot as plt

from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental
from orix import plot, io
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d, Miller

from diffsims.generators.simulation_generator import SimulationGenerator
import diffpy

In [ ]:
hs.set_log_level("ERROR")

## Load data

In [ ]:
signal = hs.load("graphite_iron_200x200.hspy", lazy=True)

In [ ]:
signal.axes_manager

In [ ]:
signal.axes_manager[0].offset = 0
signal.axes_manager[1].offset = 0

In [ ]:
signal

In [ ]:
signal.plot(cmap="magma_r", gamma=0.5)

## Center diffraction patterns

In [ ]:
s_shifts = signal.get_direct_beam_position(method="blur", sigma=5, half_square_width=20)

In [ ]:
s_shifts.compute()

In [ ]:
s_linear_plane = s_shifts.get_linear_plane()

In [ ]:
s_linear_plane.plot();

In [ ]:
signal.center_direct_beam(shifts=s_linear_plane, inplace=True)

In [ ]:
signal.plot(cmap="magma_r", gamma=0.5)

## Calibrate dataset

In [ ]:
# Calibrate navigation axes
signal.axes_manager[0].name = "x"
signal.axes_manager[0].scale = 3.704
signal.axes_manager[0].units = "nm"
signal.axes_manager[1].name = "y"
signal.axes_manager[1].scale = 3.704
signal.axes_manager[1].units = "nm"
signal.set_diffraction_calibration(0.013388)

In [ ]:
signal.plot(cmap="magma_r", gamma=0.5, scalebar_color="k")

## Crop dataset

In [ ]:
signal2 = signal.inav[:, 10:20]

In [ ]:
signal2

In [ ]:
signal2.compute()

In [ ]:
signal2.plot(cmap="magma_r", gamma=0.5, scalebar_color="k")

## Background subtraction

In [ ]:
signal2.change_dtype("u2")

In [ ]:
# Transform to polar coordinates
pol = signal2.get_azimuthal_integral2d(
    npt=100,        # Number of pixels in the radial direction
    npt_azim=360,   # Number of pixels in the azimuthal direction
    mean=False,      # Calculate mean intensity of each pixel
)

In [ ]:
pol.plot(cmap="magma_r", gamma=0.5)

In [ ]:
pol.subtract_diffraction_background(
    method="radial median", # Which background subtraction method to use. Different methods are available for polar and cartesian datasets
    inplace=True,           # Whether to modify the data in-place or return a modified copy
    )

In [ ]:
pol

In [ ]:
pol.plot(cmap="magma_r", gamma=0.5)

## Template library

In [ ]:
# Graphite orientations
angular_resolution = 1
graphite = Phase.from_cif("graphite.cif")
graphite.name = "Graphite"
graphite.color = "tab:orange"
grots = get_sample_reduced_fundamental(
    angular_resolution,             # Resolution of euler angles, in degrees
    point_group=graphite.point_group,  # Symmetry of the orientations
    )
grots.size

In [ ]:
# Ferrite orientations
angular_resolution = 1
ferrite = Phase.from_cif("Fe_ferrite.cif")
ferrite.name = "Ferrite"
ferrite.color = "tab:blue"
frots = get_sample_reduced_fundamental(
    angular_resolution,             # Resolution of euler angles, in degrees
    point_group=ferrite.point_group,  # Symmetry of the orientations
    )
frots.size

In [ ]:
graphite.structure

In [ ]:
Orientation(grots, symmetry=graphite.point_group).scatter("ipf")
Orientation(frots, symmetry=ferrite.point_group).scatter("ipf")

In [ ]:
gen = SimulationGenerator(
    accelerating_voltage=200,
    minimum_intensity=1e-3,    # The minimum intensity to keep, lower intensity reflections are discarded
    precession_angle=0,         # The precession angle in degrees.
    approximate_precession=True,# Lorentzian approximation of precession, as described in https://doi.org/10.1107/S2052520619007534
    )

In [ ]:
# Simulate diffraction patterns
simulations = gen.calculate_diffraction2d(
    phase=[ferrite, graphite],                # Which phase(s) to simulate for
    rotation=[frots, grots],              # Which orientations to simulate for. Can be multiple instances corresponding to multiple phases
    reciprocal_radius=1.7,      # Max radius to consider, in reciprocal Ångström
    with_direct_beam=False,     # Whether to include the direct beam in simulations
    max_excitation_error=0.04,  # Maximal excitation error s, in reciprocal Ångström, used for rel-rod length
    )

In [ ]:
simulations.plot(size_factor=15);

## Template matching (graphite only)

In [ ]:
# apply gamma (sometimes useful)
pol_gamma = pol**0.5

In [ ]:
simulations.phases

In [ ]:
orimap = pol_gamma.get_orientation(
    simulations.iphase[1],    # The simulation bank
    frac_keep=1.0,  # Control the fraction of the template bank used in full correlation.
    )

### Orientation map (graphite)

In [ ]:
orimap.plot_over_signal(signal2, cmap="magma_r", gamma=0.5)

In [ ]:
oris = orimap.to_single_phase_orientations().squeeze()

In [ ]:
graphite_color_key = plot.IPFColorKeyTSL(graphite.point_group, direction=Vector3d.zvector())

In [ ]:
graphite_color_key.plot()

In [ ]:
graphite_colors = graphite_color_key.orientation2color(oris)

In [ ]:
graphite_colors.shape

In [ ]:
fig, axs = plt.subplots()
axs.set_title("no gamma")
axs.imshow(graphite_colors)

### CrystalMap (graphite)

In [ ]:
xmap = orimap.to_crystal_map()

In [ ]:
xmap

In [ ]:
xmap.plot()

In [ ]:
graphite_colors = graphite_color_key.orientation2color(xmap["Graphite"].orientations)

In [ ]:
graphite_colors.shape

In [ ]:
xmap["Graphite"].plot(graphite_colors[:,0,:])

In [ ]:
io.save(filename="only_graphite_indexed.h5", object2write=xmap)

### Find zone axis

In [ ]:
rot1 = orimap.inav[30,5].to_rotation()  # graphite (blue)

In [ ]:
rot1.to_euler(degrees=True)

In [ ]:
zavec = Orientation(rot1, graphite.point_group) * Vector3d.zvector()

In [ ]:
za = Miller(xyz=zavec.data, phase=graphite)
za.coordinate_format="UVTW"
za.round()

## Template matching, 2 phases

In [ ]:
# Might need to be run twice to work correctly
borimap = pol_gamma.get_orientation(
    simulations,    # The simulation bank
    frac_keep=1.0,  # Control the fraction of the template bank used in full correlation.
    )

In [ ]:
borimap.compute()

### CrystalMap (2 phases)

In [ ]:
xmap2 = borimap.to_crystal_map()

In [ ]:
xmap2

In [ ]:
xmap2.plot()

In [ ]:
graphite_color_key = plot.IPFColorKeyTSL(graphite.point_group, direction=Vector3d.zvector())
ferrite_color_key = plot.IPFColorKeyTSL(ferrite.point_group, direction=Vector3d.zvector())

In [ ]:
ferrite_colors = ferrite_color_key.orientation2color(xmap2["Ferrite"].orientations)
graphite_colors = graphite_color_key.orientation2color(xmap2["Graphite"].orientations)

In [ ]:
xmap2["Ferrite"].plot(ferrite_colors[:,0,:])
xmap2["Graphite"].plot(graphite_colors[:,0,:])